# ODEs with Scipy

In [75]:
#%matplotlib notebook
#%matplotlib inline
import numpy as np
from scipy.integrate import odeint
from matplotlib import pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

## Example 1-Simple sdof oscillator

Consider the following oscillator

$$M\ddot U(t)+C\dot U(t)+KU(t)=F_0S_{\omega t}$$

Perform a change of variables to convert the 2-nd order ODE into a system of first order ODEs:

$$\dot U(t)= V(t)$$

$$\dot V(t)=\frac1M \lbrack F_0S_{\omega t}-CV(t)-KU(t)\rbrack$$

and place these ODEs into a callable function:

In [76]:
def model(z , t , M , C , K , f_0 , ome):
    """
    z would be the solution vector
    dzdt stores the system of ODEs ready for integration.
    """
    U = z[0]
    V = z[1]
    f_t  = force(f_0 , ome , t)
    dUdt = V
    dVdt = (1/M)*(f_t - C*V - K*U)
    dzdt = [dUdt , dVdt]
    
    return dzdt

In [77]:
def force(f_0 , ome , t):
    """
    
    """
    f_t = f_0*np.sin(ome*t)
    return f_t

In [78]:
def sdof_integrate(M , K , ome , C , f_0):
    """
    M   :Mass
    C   :Dashpot coefficient
    K   :Stiffness
    f_0 :Amplitude of the applied force
    ome :Frequency of applied force
    
    """

    y0 = [0.0 , 0.0]                 # Initial conditions
    t  = np.linspace(0 , 20 , 1001)    #Time span
    sol  = odeint(model , y0 , t , args=(M , C , K , f_0 , ome))
    
    U_ref = sol[:, 0]
    V_ref = sol[: , 1]
#
    fig1 = plt.figure(figsize=(12,6))
    axes1= fig1.add_axes([0.1,0.1,0.8,0.8] )
    axes1.set_ylabel('$U$')
    axes1.set_xlabel('$t[s]$')
    axes1.plot(t, U_ref )
    plt.grid()
    
    F_r = K*U_ref + C*V_ref
    F_s = K*U_ref
    F_v = C*V_ref
        
    fig2 = plt.figure(figsize=(12,6))
    axes2= fig2.add_axes([0.1,0.1,0.8,0.8] )
    axes2.set_ylabel('$U$')
    axes2.set_xlabel('$t[s]$')
    axes2.plot(U_ref , F_r )    
    axes2.plot(U_ref , F_s )
    axes2.plot(U_ref , F_v )
    plt.grid()
        
    return

In [79]:
interact(sdof_integrate , M = (1, 10) , K = (20 , 100) , ome =(0.5 , 30.0) , C = (0.1 , 0.5) , f_0 = (1.0 , 10.0))

interactive(children=(IntSlider(value=5, description='M', max=10, min=1), IntSlider(value=60, description='K',…

<function __main__.sdof_integrate(M, K, ome, C, f_0)>

In [80]:
def misses(F_0 , fac , lmda , x):
    """
    Constitutive law for the Misses truss, This is piecewise
    continuous function.
    F_0 : force amplitude
    fac : number of wavelengths at which the constant slope phase starts
    lmda: Wavelength
    x   : Displacement
    """
    dx = lmda/100
    df = F_0*np.sin((2*np.pi/lmda)*dx)
    k = df/dx
    if x > -fac*lmda and x < fac*lmda:
        F = F_0*np.sin((2*np.pi/lmda)*x)
    else:
        if x<= -fac*lmda:
            F = k*(x + fac*lmda)
        else:
            F = k*(x - fac*lmda)
    return F

In [81]:
def mis_dis(F_0 , fac , lmda):
#    F_0   = 10.0    #Force amplitude for the Misses truss
#    fac   = 1.0    #Piecewise continous at fac*lambda
#    lmda  = 0.5   #Wavelength for the Misses truss F-d relationship
#
    x  = np.linspace(-2*lmda , 2*lmda, 1001) #Assumed displacements span
    n = len(x)
    FM = np.zeros(n)
#
    for i in range(n):
        FM[i] = misses(F_0 , fac , lmda , x[i])
        
        
    fig3 = plt.figure(figsize=(12,6))
    axes3= fig3.add_axes([0.1,0.1,0.8,0.8] )
    axes3.set_ylabel('$U$')
    axes3.set_xlabel('$t[s]$')
    axes3.plot(x , FM )    
    plt.grid()
    return

In [82]:
interact(mis_dis , F_0 = (1, 10) , fac = (0.5 , 1.0) , lmda =(0.1 , 0.5))

interactive(children=(IntSlider(value=5, description='F_0', max=10, min=1), FloatSlider(value=0.75, descriptio…

<function __main__.mis_dis(F_0, fac, lmda)>

# System with  added dissipation device

In [83]:
def dissip_device(z , t , M , C , K , f_0 , ome , m , c , k , F_0 , fac , lmda):
    """
    z would be the solution vector
    dzdt stores the system of ODEs ready for integration.
    """
    U = z[0]
    V = z[1]
    u = z[2]
    v = z[3]
#
    F_AP  = force(f_0 , ome , t)
    F_VM  = misses(F_0 , fac , lmda ,U-u)
#
    dUdt = V
    dVdt = (1/M)*(F_AP - C*V - K*U-F_VM)
    dudt = v
    dvdt = (1/m)*(F_VM - k*u - c*v)
#
    dzdt = [dUdt , dVdt , dudt , dvdt]
    
    return dzdt

In [84]:
def device_integrate(M , K , ome , C , f_0 , c , k , F_0 , fac , lmda):
    """
    M   :Mass
    C   :Dashpot coefficient
    K   :Stiffness
    f_0 :Amplitude of the applied force
    ome :Frequency of applied force
    
    """
    m   = 1.0e-3 

    y0 = [0.0 , 0.0 , 0.0 , 0.0]                  # Initial conditions
    t  = np.linspace(0 , 20 , 1001)    #Time span
    sol  = odeint(dissip_device , y0 , t , args =(M , C , K , f_0 , ome , m , c , k , F_0 , fac , lmda))
    
    U_dev = sol[:, 0] 
    V_dev = sol[: , 1]
#
    fig4 = plt.figure(figsize=(12,6))
    axes4= fig4.add_axes([0.1,0.1,0.8,0.8] )
    axes4.set_ylabel('$U$')
    axes4.set_xlabel('$t[s]$')
    axes4.plot(t, U_dev )
    plt.grid()
    
#    F_r = K*U_dev + C*V_dev
#    F_s = K*U_ref
#    F_v = C*V_ref
#        
#    fig2 = plt.figure(figsize=(12,6))
#    axes2= fig2.add_axes([0.1,0.1,0.8,0.8] )
#    axes2.set_ylabel('$U$')
#    axes2.set_xlabel('$t[s]$')
#    axes2.plot(U_ref , F_r )    
#    axes2.plot(U_ref , F_s )
#    axes2.plot(U_ref , F_v )
#    plt.grid()
        
    return

In [85]:
interact(device_integrate , M = (1, 10) , K = (20 , 100) , ome =(0.5 , 30) , C = (0.1 , 0.5) , f_0 = (1.0 , 10.0) ,
         c =(0.0 , 0.5) , k=(20.0 , 100.0) , F_0 = (1.0, 10.0) , fac = (0.5 , 1.0) , lmda =(0.1 , 1.0))

interactive(children=(IntSlider(value=5, description='M', max=10, min=1), IntSlider(value=60, description='K',…

<function __main__.device_integrate(M, K, ome, C, f_0, c, k, F_0, fac, lmda)>